# Advent of Code

## Day 1

Find the top three Elves carrying the most Calories. How many Calories are those Elves carrying in total?

In [1]:
from itertools import groupby

topk = 3

with open("data/day1.txt", "r") as f:
    data = [l.strip() for l in f.readlines()]
    
sum(sorted(sum(map(int, v)) for k, v in groupby(data, key=lambda l: l != '') if k)[-topk:])

212520

## Day 2

What would your total score be if everything goes exactly according to your strategy guide?

In [2]:
with open("data/day2.txt", "r") as f:
    data = [l.strip().split(' ') for l in f.readlines()]

first_col = {k: v for v, k in enumerate('ABC')}
second_col = {k: v for v, k in enumerate('XYZ')}
mapping = list(map(lambda x: (first_col[x[0]], second_col[x[1]]), data))

shape_points = map(lambda x: 1 + x[1], mapping)
outcome_points = map(lambda y : 3 * (y + 1), [v if abs(v) != 2 else -v // 2 for v in map(lambda x: x[1] - x[0], mapping)])
sum(shape_points) + sum(outcome_points)

9759

In [3]:
shape_points = map(lambda x: 1 + ((x[0] + x[1] - 1) % 3), mapping)
outcome_points = map(lambda x: 3 * second_col[x[1]], data)
sum(shape_points) + sum(outcome_points)

12429